In [1]:
!pip install -q kaggle
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d jangedoo/utkface-new
!unzip -qq utkface-new.zip

 95% 314M/331M [00:02<00:00, 146MB/s]
100% 331M/331M [00:02<00:00, 127MB/s]


In [ ]:
!pip install wandb

In [5]:
cd /content/drive/MyDrive/FaceAgeRegression

/content/drive/MyDrive/FaceAgeRegression


In [6]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, Dataset
from tqdm import tqdm
from model import *
from data_loader import *
import wandb


In [7]:
wandb.init(project="Face Age Regression")
# wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
batch_size = 64
epochs = 20
config = wandb.config
config.learning_rate = 0.001
width = height = 224

In [13]:
def build_model():
  model = models.resnet50(pretrained=True)
  in_fetures = model.fc.in_features
  model.fc = nn.Linear(in_fetures, 1)

  device = torch.device("cuda")
  model = model.to(device)

  # This freezes layers 1-6 in the total 10 layers of Resnet50
  ct = 0
  for child in model.children():
      ct += 1
      if ct < 7:
          for param in child.parameters():
              param.requires_grad = False
  return model

def build_dataset(batch_size):
  
  train_data, _ = data_loader("/content/crop_part1")

  torch.manual_seed(0)

  train_dataset_size = int(0.8 * len(train_data))
  test_dataset_size = len(train_data) - train_dataset_size

  train_data, test_data = torch.utils.data.random_split(train_data, [train_dataset_size, test_dataset_size])
  train_data = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
  test_data = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)
  return train_data, test_data

In [14]:
train_data = build_dataset(batch_size)
model = build_model()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = nn.MSELoss()

In [16]:
device = torch.device("cuda")


In [17]:
# train

model.train()

for epoch in range(epochs):
  train_loss = 0.0
  for images, labels in tqdm(train_data):

    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    images = images.float()
    preds = model(images)

    loss = loss_function(preds,  labels.float())
    loss.backward()

    optimizer.step()

    train_loss += loss

  total_loss = train_loss / len(train_data)
  print(f"Epoch: {epoch+1}, Loss: {total_loss}")

  wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
                              })

100%|██████████| 95/95 [01:14<00:00,  1.28it/s]


Epoch: 1, Loss: 431.1103515625


100%|██████████| 95/95 [01:14<00:00,  1.27it/s]


Epoch: 2, Loss: 254.10838317871094


100%|██████████| 95/95 [01:14<00:00,  1.27it/s]


...

Epoch: 18, Loss: 78.04642486572266


100%|██████████| 95/95 [01:13<00:00,  1.29it/s]


Epoch: 19, Loss: 74.69387817382812


100%|██████████| 95/95 [01:13<00:00,  1.29it/s]

Epoch: 20, Loss: 72.9371566772461


In [18]:
torch.save(model.state_dict(), "/content/drive/MyDrive/FaceAgeRegression/age_regression_mse.pth")